In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [44]:
data_d <- '/oak/stanford/groups/mrivas/users/gmcinnes/statin_risk_score'
phe_all_file <- file.path(data_d, 'statin_PRS_all.phe')
phe_minimum_file <- file.path(data_d, 'statin_PRS_snpnet.phe')


In [2]:
GWAS_covar_f <- '/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification_w24983_20200313/ukb24983_GWAS_covar.20200313.phe'
phe_fs <- c(
    '/oak/stanford/groups/mrivas/users/gmcinnes/atorvastatin_v_hc_and_Ostatin.phe',
    '/oak/stanford/groups/mrivas/users/gmcinnes/simvastatin_v_hc_and_Ostatin.phe'
)


In [34]:
phe_fs %>% lapply(function(phe_f){
    fread(
        phe_f,
        col.names=c('FID', 'IID', str_replace(basename(phe_f), '.phe$' , '')),
        colClasses=c('character', 'character', 'integer')
    )
}) %>%
reduce(function(x, y){
    inner_join(x, y, by=c('FID', 'IID'))
}) %>% 
inner_join(
    fread(
        GWAS_covar_f,
        colClasses=c('FID'='character', 'IID'='character', 'split'='character')
    ), by=c('FID', 'IID')
) -> phe_df


In [36]:
phe_df %>% count(atorvastatin_v_hc_and_Ostatin, simvastatin_v_hc_and_Ostatin)

atorvastatin_v_hc_and_Ostatin,simvastatin_v_hc_and_Ostatin,n
<int>,<int>,<int>
1,1,4773
1,2,43994
2,1,15379
2,2,941


In [38]:
phe_df %>% count(population)

population,n
<chr>,<int>
african,707
e_asian,88
e_asian_outlier,51
non_british_white,2810
s_asian,1703
s_asian_outlier,15
white_british,44774
NA,14939


In [42]:
phe_df %>% count(split)

split,n
<chr>,<int>
test,8971
train,31368
val,4437
NA,20311


In [46]:
colnames(phe_df)

[1] "FID"                           "IID"                          
 [3] "atorvastatin_v_hc_and_Ostatin" "simvastatin_v_hc_and_Ostatin" 
 [5] "population"                    "split"                        
 [7] "age"                           "age0"                         
 [9] "age1"                          "age2"                         
[11] "age3"                          "sex"                          
[13] "BMI"                           "N_CNV"                        
[15] "LEN_CNV"                       "Array"                        
[17] "PC1"                           "PC2"                          
[19] "PC3"                           "PC4"                          
[21] "PC5"                           "PC6"                          
[23] "PC7"                           "PC8"                          
[25] "PC9"                           "PC10"                         
[27] "PC11"                          "PC12"                         
[29] "PC13"                          "PC14"                         
[31] "PC15"                          "PC16"                         
[33] "PC17"                          "PC18"                         
[35] "PC19"                          "PC20"                         
[37] "PC21"                          "PC22"                         
[39] "PC23"                          "PC24"                         
[41] "PC25"                          "PC26"                         
[43] "PC27"                          "PC28"                         
[45] "PC29"                          "PC30"                         
[47] "PC31"                          "PC32"                         
[49] "PC33"                          "PC34"                         
[51] "PC35"                          "PC36"                         
[53] "PC37"                          "PC38"                         
[55] "PC39"                          "PC40"                         
[57] "Global_PC1"                    "Global_PC2"                   
[59] "Global_PC3"                    "Global_PC4"                   
[61] "Global_PC5"                    "Global_PC6"                   
[63] "Global_PC7"                    "Global_PC8"                   
[65] "Global_PC9"                    "Global_PC10"                  
[67] "Global_PC11"                   "Global_PC12"                  
[69] "Global_PC13"                   "Global_PC14"                  
[71] "Global_PC15"                   "Global_PC16"                  
[73] "Global_PC17"                   "Global_PC18"                  
[75] "Global_PC19"                   "Global_PC20"                  
[77] "Global_PC21"                   "Global_PC22"                  
[79] "Global_PC23"                   "Global_PC24"                  
[81] "Global_PC25"                   "Global_PC26"                  
[83] "Global_PC27"                   "Global_PC28"                  
[85] "Global_PC29"                   "Global_PC30"                  
[87] "Global_PC31"                   "Global_PC32"                  
[89] "Global_PC33"                   "Global_PC34"                  
[91] "Global_PC35"                   "Global_PC36"                  
[93] "Global_PC37"                   "Global_PC38"                  
[95] "Global_PC39"                   "Global_PC40"

In [47]:
phe_df %>% 
fwrite(phe_all_file, sep='\t', na = "NA", quote=F)


In [51]:
phe_df %>% 
filter(split %in% c('train', 'val')) %>%
# rename(
#     'atorvastatin' = 'atorvastatin_v_hc_and_Ostatin',
#     'simvastatin'  = 'simvastatin_v_hc_and_Ostatin'
# ) %>%
select(
    FID, IID, age, sex, paste0('PC', 1:10), split,
#     atorvastatin, simvastatin
    atorvastatin_v_hc_and_Ostatin, simvastatin_v_hc_and_Ostatin
) %>%
fwrite(phe_minimum_file, sep='\t', na = "NA", quote=F)


In [52]:
phe_minimum_file

[1] "/oak/stanford/groups/mrivas/users/gmcinnes/statin_risk_score/statin_PRS_snpnet.phe"